In [1]:
import numpy as np
import pandas as pd

In [2]:
SP = pd.read_csv('S&P_data.csv', parse_dates=[0])
SP = SP.loc[20:]
SP.reset_index(inplace=True)

__First Set__

In [5]:
n = SP.shape[0]
SP['Rtn'] = np.empty(n)

for i in range(1, n):
    SP.loc[i, 'Rtn'] = np.log(SP['spindx'][i] / SP['spindx'][i-1])
    
SP.loc[0, 'Rtn'] = np.mean(SP['Rtn'][1:])

In [6]:
SP['(Rtn-Rbar)^2'] = np.empty(n)

for i in range(0, n):
    SP.loc[i, '(Rtn-Rbar)^2'] = (SP['Rtn'][i] - SP['Rtn'][0]) ** 2

In [8]:
gamma = 0.00000102638439016707
alpha = 0.067553161383582
beta = 0.925509995984242
alpha_beta = alpha + beta
LT_vol = np.sqrt(250*(gamma)/(1-alpha_beta))

In [9]:
SP['Cond Var'] = np.empty(n)
SP.loc[0, 'Cond Var'] = SP['Rtn'][0] ** 2

for i in range(1, n):
    SP.loc[i, 'Cond Var'] = gamma + alpha * SP['(Rtn-Rbar)^2'][i-1] + beta * SP['Cond Var'][i-1]

In [10]:
SP['Log L'] = np.empty(n)
SP.loc[0, 'Log L'] = '--'

for i in range(1, n):
    SP.loc[i, 'Log L'] = -(np.log(np.sqrt(SP['Cond Var'][i])))-0.5*(SP['(Rtn-Rbar)^2'][i]/SP['Cond Var'][i])

In [11]:
SP['GARCH vol'] = np.empty(n)
SP.loc[0, 'GARCH vol'] = '--'

for i in range(1, n):
    SP.loc[i, 'GARCH vol'] = np.sqrt(SP['Cond Var'][i] * 250)

In [12]:
LogL = np.sum(SP['Log L'][1:])
LogL

13848.676835779945

__Second Set__

In [13]:
SP_2 = pd.read_csv('S&P_data.csv', parse_dates=[0])
SP_2 = SP_2.loc[20:]
SP_2.reset_index(inplace=True)

gamma = 0.000000360946440498536
alpha = 0.0978672516991889
beta = 0.898840462526082
Lambda = 0.0037694281717221
alpha_beta = alpha + beta
LT_vol = np.sqrt(250*(gamma + (Lambda ** 2) * alpha) / (1 - alpha_beta))

In [14]:
SP_2['(Rtn-Rbar-l)^2'] = np.empty(n)

for i in range(0, n):
    SP_2.loc[i, '(Rtn-Rbar-l)^2'] = (SP['Rtn'][i] - SP['Rtn'][0] - Lambda) ** 2

In [15]:
SP_2['Cond Var'] = np.empty(n)
SP_2.loc[0, 'Cond Var'] = SP['Rtn'][0] ** 2

for i in range(1, n):
    SP_2.loc[i, 'Cond Var'] = gamma + alpha * SP_2['(Rtn-Rbar-l)^2'][i-1] + beta * SP_2['Cond Var'][i-1]

In [16]:
SP_2['Log L'] = np.empty(n)
SP_2.loc[0, 'Log L'] = '--'

for i in range(1, n):
    SP_2.loc[i, 'Log L'] = -(np.log(np.sqrt(SP_2['Cond Var'][i])))-0.5*(SP['(Rtn-Rbar)^2'][i]/SP_2['Cond Var'][i])

In [17]:
SP_2['AGARCH vol'] = np.empty(n)
SP_2.loc[0, 'AGARCH vol'] = '--'

for i in range(1, n):
    SP_2.loc[i, 'AGARCH vol'] = np.sqrt(SP_2['Cond Var'][i] * 250)

In [18]:
LogL_2 = np.sum(SP_2['Log L'][1:])
LogL_2

13887.452857622384

__GARCH and AGARCH Returns__

In [19]:
Returns = pd.read_csv('S&P_data.csv', parse_dates=[0])
Returns = Returns.loc[20:]
Returns.reset_index(inplace=True)

In [20]:
Returns['GARCH Rtn'] = np.empty(n)
Returns.loc[0, 'GARCH Rtn'] = SP['Rtn'][0]

for i in range(1, n):
    Returns.loc[i, 'GARCH Rtn'] = SP['Rtn'][i] * (SP['GARCH vol'][n-1] / SP['GARCH vol'][i])

In [21]:
Returns['AGARCH Rtn'] = np.empty(n)
Returns.loc[0, 'AGARCH Rtn'] = SP['Rtn'][0]

for i in range(1, n):
    Returns.loc[i, 'AGARCH Rtn'] = SP['Rtn'][i] * (SP_2['AGARCH vol'][n-1] / SP_2['AGARCH vol'][i])

__Historical VaR Estimates, 31 March 2008__

In [22]:
Quantiles = [0.10, 1, 5, 10]
VaR = pd.DataFrame({'Quantile': ['0.10%', '1%', '5%', '10%'], 'Rtn': [0.0, 0.0, 0.0, 0.0], 'GARCH Rtn': [0.0, 0.0, 0.0, 0.0], \
                    'AGARCH Rtn': [0.0, 0.0, 0.0, 0.0]})

In [23]:
for i in range(0, 4):
    VaR.loc[i, 'Rtn'] = -np.percentile(SP['Rtn'], Quantiles[i])
    
for i in range(0, 4):
    VaR.loc[i, 'GARCH Rtn'] = -np.percentile(Returns['GARCH Rtn'], Quantiles[i])
    
for i in range(0, 4):
    VaR.loc[i, 'AGARCH Rtn'] = -np.percentile(Returns['AGARCH Rtn'], Quantiles[i])

In [24]:
VaR

,Quantile,Rtn,GARCH Rtn,AGARCH Rtn
0,0.10%,0.048355,0.075500,0.070356
1,1%,0.028293,0.041105,0.042780
2,5%,0.017755,0.026568,0.027946
3,10%,0.012714,0.020448,0.021110


In [25]:
gamma = 0.00000102638439016707
alpha = 0.067553161383582
beta = 0.925509995984242

In [26]:
%store SP
%store SP_2
%store Returns
%store VaR

Stored 'SP' (DataFrame)
Stored 'SP_2' (DataFrame)
Stored 'Returns' (DataFrame)
Stored 'VaR' (DataFrame)
